In [3]:
import numpy as np

These are the steps:

1. Calculate the time interval (dt) and the up (u) and down (d) factors based on the volatility and time interval.

2. Compute the risk-neutral probability p using the interest rate, up, and down factors.

3. Create the binomial price tree, filling it with the possible prices of the underlying asset at each time step.

4. Calculate the option value at the last time step (the expiration date) using the price tree.

5. Traverse the tree backward, comparing the value of exercising the option with the value of holding it (update the option tree with the maximum of these two values).

6. The estimated option price is the value at the root of the option tree (i.e., option_tree[0, 0]).

In [1]:
def american_option_pricer(spot, strike, rate, vol, expiry, steps, option_type):
    # Calculate the time interval and the up and down factors
    dt = expiry / steps
    u = np.exp(vol * np.sqrt(dt))
    d = 1 / u

    # Calculate the risk-neutral probability
    p = (np.exp(rate * dt) - d) / (u - d)

    # Create the binomial price tree
    price_tree = np.zeros((steps + 1, steps + 1))
    for i in range(steps + 1):
        price_tree[i, -1] = spot * (u ** (steps - i)) * (d**i)

    # Calculate the option value at each node
    option_tree = np.zeros_like(price_tree)
    if option_type.lower() == "call":
        option_tree[:, -1] = np.maximum(price_tree[:, -1] - strike, 0)
    elif option_type.lower() == "put":
        option_tree[:, -1] = np.maximum(strike - price_tree[:, -1], 0)
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    # Traverse the tree backward to find the option price today
    for t in range(steps - 1, -1, -1):
        for i in range(t + 1):
            exercise = 0
            if option_type.lower() == "call":
                exercise = price_tree[i, t] - strike
            elif option_type.lower() == "put":
                exercise = strike - price_tree[i, t]

            hold = np.exp(-rate * dt) * (
                p * option_tree[i, t + 1] + (1 - p) * option_tree[i + 1, t + 1]
            )
            option_tree[i, t] = np.maximum(exercise, hold)

    return option_tree[0, 0]

In [6]:
option_price = american_option_pricer(
    spot=55.0,
    strike=50.0,
    rate=0.05,
    vol=0.3,
    expiry=1.0,
    steps=100,
    option_type="call",
)
option_price


10.542838907149742